### 如何自己实现一个事件驱动回测

如果自己要设计实现一个事件驱动回测系统，可能要包含以下几个基本组件：

- 事件：事件是系统功能的基本单元。诸如，如“行情”、“信号”、“订单”、“”等，该类型决定了它在事件循环中如何处理。

- 事件队列：事件队列是一个内存中的Python队列对象，存储了所有触发并待处理的事件。

- 数据处理：一个抽象基类，为处理历史或实时市场数据提供接口。

- 策略：一个抽象基类，它提供了接收市场数据并生成相应信号事件的接口。

- 执行引擎：模拟与经纪公司的连接。从队列中接收订单事件并执行它们。一旦订单被执行，处理器会创建成交事件，描述实际成交的内容，包括费用、佣金和滑点等。

- 回测：所有这些组件都被封装在一个事件循环中，该循环正确处理所有事件类型，并将它们路由到适当的组件。

#### 事件

In [10]:
##### 所有事件的基类
class Event(object): 
    """Event is base class providing an interface for all subsequent (inherited) events"""
    pass

##### 行情事件,行情来的时候触发
class QuoteEvent(Event): 
    def __init__(self):
        self.type = 'QUOTE'
        
##### 信号事件，策略触发，买还是卖
class SignalEvent(Event): 
    def __init__(self, strategy_id, symbol, datetime, signal_type, strength): 
        # 事件的类型
        self.type = 'SIGNAL'
        # 策略ID
        self.strategy_id = strategy_id
        # 股票代码
        self.symbol = symbol
        # 触发时间
        self.datetime = datetime
        # 信号的具体类型
        self.signal_type = signal_type
        #self.strength = strength
        
##### 订单，引擎触发
class OrderEvent(Event):
    def __init__(self, symbol, order_type, quantity, direction):
        # 事件的类型
        self.type = 'ORDER'
         # 股票代码
        self.symbol = symbol
         # 订单类型
        self.order_type = order_type
        # 订单数量
        self.quantity = quantity
        # 订单方向
        self.direction = direction
    def print_order(self): 
        """ Outputs the values within the Order. """
        print(
        "Order: Symbol=%s, Type=%s, Quantity=%s, Direction=%s" %
        (self.symbol, self.order_type, self.quantity, self.direction) )

        
##### 成本费用，引擎触发
class CostEvent(Event):
    def __init__(self, timeindex, symbol, exchange, quantity, direction, fill_cost, commission=None):
        # 事件的类型
        self.type =  'COST'
        # 费用产生时间
        self.timeindex = timeindex
        # 股票代码
        self.symbol = symbol
        # 交易所
        self.exchange = exchange
        # 数量
        self.quantity = quantity
        # 方向
        self.direction = direction
        #self.fill_cost = fill_cost
        # Calculate commission
        if commission is None:
            self.commission = self.calculate_ib_commission()
        else:
            self.commission = commission
    def calculate_ib_commission(self):
        # 假设默认费用为千分之一
        return 0.001

#### 数据